In [ ]:
import pandas as pd
import numpy as np
import keras

In [ ]:
#keras.layers.LocallyConnected1D(filters, kernel_size, strides=1, padding='valid', data_format=None, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

Arguments

filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).

kernel_size: An integer or tuple/list of a single integer, specifying the length of the 1D convolution window.

strides: An integer or tuple/list of a single integer, specifying the stride length of the convolution. Specifying any 
stride value != 1 is incompatible with specifying any dilation_rate value != 1.

padding: Currently only supports "valid" (case-insensitive).  "same" may be supported in the future.

data_format: String, one of channels_first, channels_last.

activation: Activation function to use (see activations). If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).

use_bias: Boolean, whether the layer uses a bias vector.

kernel_initializer: Initializer for the kernel weights matrix (see initializers).

bias_initializer: Initializer for the bias vector (see initializers).

kernel_regularizer: Regularizer function applied to the kernel weights matrix (see regularizer).

bias_regularizer: Regularizer function applied to the bias vector (see regularizer).

activity_regularizer: Regularizer function applied to the output of the layer (its "activation"). (see regularizer).

kernel_constraint: Constraint function applied to the kernel matrix (see constraints).

bias_constraint: Constraint function applied to the bias vector (see constraints).

Input shape: 3D tensor with shape - (batch_size, steps, input_dim)

Output shape; 3D tensor with shape - (batch_size, new_steps, filters) steps value might have changed due to padding or strides

In [2]:
# # Example:
# # apply a unshared weight convolution 1d of length 3 to a sequence with
# # 10 timesteps, with 64 output filters
# model = Sequential()
# model.add(LocallyConnected1D(64, 3, input_shape=(10, 32)))
# # now model.output_shape == (None, 8, 64)
# # add a new conv1d on top
# model.add(LocallyConnected1D(32, 3))
# # now model.output_shape == (None, 6, 32)